<a href="https://colab.research.google.com/github/pythonuzgit/elmurodov/blob/master/Convolutional%20neural%20network/Image_Classification_model_using_PyTorch_with_applescablds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!kaggle datasets download -d projectlzp201910094/applescablds

100% 3.21G/3.22G [00:56<00:00, 22.2MB/s]
100% 3.22G/3.22G [00:56<00:00, 61.3MB/s]


In [ ]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.transforms import transforms
import torchvision.transforms as T
import torchvision.models as models
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

import os
from PIL import Image



Clean data an remove non images files

In [ ]:
path  = '/content/AppleScabLDs'

for folder in os.listdir(path):
  for img_file in os.listdir(os.path.join(path, folder)):
    img_file = os.path.join(path, folder, img_file)

    try:
      img = Image.open(img_file)
      if img.mode != 'RGB':
        os.remove(img_file)

    except:
      os.remove(img_file)    

Data Preprocessing and convert to tensor

In [ ]:
transform = transforms.Compose([
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5],
                                                     [0.5])
])

dataset = datasets.ImageFolder('/content/AppleScabLDs', transform = transform)

dataset_len = len(dataset)

train_len, test_len = dataset_len - 300, 300

train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])

batch_size = 200

#train and test dataloader
train_set = DataLoader(dataset = train_set, shuffle = True, batch_size = batch_size)
test_set = DataLoader(dataset = test_set, shuffle = True, batch_size = batch_size)

Checking device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cpu


Exploring Dataset

In [ ]:
classes = os.listdir(path)

print('Total classes:', len(classes))

Total classes: 2


Build Model

In [ ]:
class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()

    self.pool = nn.MaxPool2d(2,2)
    self.dropout = nn.Dropout(p = 0.2)

    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = 4)
    self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 12, kernel_size = 4)
    self.conv3 = nn.Conv2d(in_channels = 12, out_channels = 14, kernel_size = 4)
    self.conv4 = nn.Conv2d(in_channels = 14, out_channels = 16, kernel_size = 4)
    self.conv5 = nn.Conv2d(in_channels = 16, out_channels = 20, kernel_size = 4)

    
    self.fc1 = nn.Linear(in_features = 20 * 4 * 4, out_features = 250)
    self.fc2 = nn.Linear(in_features = 250, out_features = 200)
    self.fc3 = nn.Linear(in_features = 200, out_features = 50)
    self.fc4 = nn.Linear(in_features = 50, out_features = 10)
    self.fc5 = nn.Linear(in_features = 10, out_features = 5)



  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = self.pool(F.relu(self.conv4(x)))
    x = self.pool(F.relu(self.conv5(x)))



    x = x.reshape(-1, 20 * 4 * 4)
    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout(F.relu(self.fc2(x)))
    x = self.dropout(F.relu(self.fc3(x)))
    x = self.dropout(F.relu(self.fc4(x)))
    x = self.fc5(x)

    return x


net = Model().to(device)

print(net)


Model(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (conv1): Conv2d(3, 6, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(4, 4), stride=(1, 1))
  (conv3): Conv2d(12, 14, kernel_size=(4, 4), stride=(1, 1))
  (conv4): Conv2d(14, 16, kernel_size=(4, 4), stride=(1, 1))
  (conv5): Conv2d(16, 20, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=250, bias=True)
  (fc2): Linear(in_features=250, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=10, bias=True)
  (fc5): Linear(in_features=10, out_features=5, bias=True)
)


Define Loss and optimizer functions

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.001, weight_decay = 1e-5)

Train Data

In [ ]:
net.train()

for epoch in range(25):
  total_correct = 0.0
  running_loss = 0.0
  for i, (inputs, labels) in enumerate(train_set):
    inputs, labels = inputs.to(device), labels.to(device)
    output = net(inputs)
    output_idx = torch.argmax(output, dim = 1)
    total_correct += (labels == output_idx).sum().item()
    optimizer.zero_grad()
    loss = creterion(output, labels)
    running_loss += loss.item() * inputs.size(0)

    loss.backward()
    optimizer.step()



  print(f'Epoch: {epoch}  Loss: {running_loss/train_len}  Accuracy: {(total_correct/train_len) * 100}%') 


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0  Loss: 1.619990551069881  Accuracy: 17.865168539325843%
Epoch: 1  Loss: 1.6036869461616774  Accuracy: 17.865168539325843%
Epoch: 2  Loss: 1.4058881020278073  Accuracy: 29.55056179775281%
Epoch: 3  Loss: 0.8208612712581506  Accuracy: 74.8314606741573%
Epoch: 4  Loss: 0.6765493978275342  Accuracy: 74.49438202247191%
Epoch: 5  Loss: 0.6616781220007478  Accuracy: 70.4494382022472%
Epoch: 6  Loss: 0.6111323733008309  Accuracy: 76.17977528089888%
Epoch: 7  Loss: 0.5945323940073506  Accuracy: 79.5505617977528%
Epoch: 8  Loss: 0.5962608188725589  Accuracy: 80.2247191011236%
Epoch: 9  Loss: 0.5885730137985744  Accuracy: 80.4494382022472%
Epoch: 10  Loss: 0.5355398902732335  Accuracy: 80.11235955056179%
Epoch: 11  Loss: 0.5779945903949524  Accuracy: 79.7752808988764%
Epoch: 12  Loss: 0.5370925423804294  Accuracy: 81.12359550561797%
Epoch: 13  Loss: 0.5404501607578792  Accuracy: 80.67415730337079%
Epoch: 14  Loss: 0.5476463028554166  Accuracy: 80.4494382022472%
Epoch: 15  Loss: 0.5279752